In [ ]:
from scipy.integrate import odeint

%env USING_RUN True

%run ./adaptive_lighting/adaptive_lighting.ipynb import AdaptiveLighting
%run ./adaptive_lighting/solarpanel.ipynb import SolarPanel
%run ./heatpump/heatpump.ipynb import HeatPump
%run ./structure/structure.ipynb import Structure
%run ./dehumidifier/dehumidifier.ipynb import Dehumidifier
%run ../crops/sweet_basil.ipynb import SweetBasil

import sys
sys.path.insert(0, '/work/greenhouse-simulator-2/')

from helpers.types import *
from helpers.conversions import *
from helpers.solar_conversions import *
from greenhouse.greenhouse import *
from greenhouse.adaptive_lighting import *

In [ ]:
class Greenhouse:
    def __init__(self, 
        time_period: s,
        # target_temp_range: [C, C],
        # target_humidity_range: [RH, RH]
    ):
        self.time_period: s = time_period
        # self.target_temp_range: [C, C] = target_temp_range
        # self.target_humidity_range: [RH, RH] = target_humidity_range

        # Init subsystems
        self.structure = Structure()
        self.dehumidifier = Dehumidifier()
        self.heatpump = HeatPump()
        self.crop = SweetBasil(
            time_period=self.time_period, 
            plants_per_barrel=self.structure.plants_per_barrel, 
            barrel_count=self.structure.barrel_count
        )
        self.light = AdaptiveLighting(time_period, self.structure, self.crop)

        # Init register to store previos period's values
        self.prev_period = {
            "CO2": 410
        }


    def run(self, timestamp, data):
        # SECTION 1: Get input from solar radiation
        # Get distribution of irradiance on different panels (solar and non-solar) of the greenhouse (factoring in sun position, tilt and azimuth angles)
        results: [W, W_per_m2] = self.structure.get_irradiance_by_panel_type(timestamp, data.solarradiation)
        solar_power_on_nonsolar_panels, irradiance_on_solar_panels = results

        # Get electrical energy generated and heat irradiated to the greenhouse
        results: [W, W, W_per_m2] = self.light.solarpanel.run(irradiance_on_solar_panels)
        solar_power_generated, solar_power_transmitted, transmitted_irradiance, transparency, _ = results
        solar_energy_generated: kWh = J_to_kWh(solar_power_generated * self.time_period)

        # Add energy from non-solar panels and portion of energy transmitted by solar panels to get total heat entering the greenhouse
        heat_transfer_rate_inside: W = solar_power_on_nonsolar_panels + solar_power_transmitted

        # Calculate energy used for lighting
        PPFD: umol_per_m2_s = irradiance_to_PPFD(transmitted_irradiance)
        PAR_inside: umol_per_m2 = PPFD * self.time_period
        lighting_results = self.light.run(timestamp, PAR_inside)

        # SECTION 2: Grow plants
        results: [mol_per_s, mol, mol_per_s, mol] = self.crop.grow(1e-6 * lighting_results["actual_PPFD_umol_per_m2_s"] * self.time_period)
        CO2_assimilation_rate, CO2_assimilated, H20_evaporation_rate, H2O_evaporated = results

        # SECTION 3: Deal with resulting CO2, water, heat
        # Calculate net energy
        total_energy_used: kWh = lighting_results["energy_used_for_lighting_kWh"] #+ energy_for_dehumidification + energy_for_heating
        total_energy_generated: kWh = solar_energy_generated
        net_energy: kWh = total_energy_generated - total_energy_used

        # Calculate CO2
        CO2_concentration: ppm = self.register_CO2(CO2_assimilated)

        return {
            "natural_PPFD": PPFD,
            "total_energy_used_kWh": total_energy_used,
            "total_energy_generated_kWh": total_energy_generated,
            "net_energy_kWh": net_energy,

            "CO2_concentration_ppm": CO2_concentration,
            "CO2_assimilation_rate_mol_per_s": CO2_assimilation_rate,

            **lighting_results,
        }

    def register_CO2(self, CO2_assimilated: mol):
        ambient_CO2: ppm = 410
        airflow: m3_per_s = 0.5

        initial_CO2_concentration: ppm = self.prev_period["CO2"]

        CO2_assimilated_per_s: mol_per_s = CO2_assimilated / self.time_period

        def CO2_model(CO2_concentration: ppm, t):
            CO2_inflow: mol_per_s = ppm_to_amount(ambient_CO2, airflow)
            CO2_outflow: mol_per_s = ppm_to_amount(CO2_concentration, airflow)
            net_change_amount: mol_per_s = CO2_inflow - CO2_assimilated_per_s - CO2_outflow
            net_change_concentration: ppm_per_s = amount_to_ppm(net_change_amount, self.structure.volume)

            return net_change_concentration


        CO2_concentrations_in_period: list[ppm] = odeint(CO2_model, initial_CO2_concentration, np.linspace(0, self.time_period, 5))
        new_CO2_concentration = CO2_concentrations_in_period[-1][0]

        if new_CO2_concentration < 0:
            new_CO2_concentration = 0

        self.prev_period["CO2"] = new_CO2_concentration

        return new_CO2_concentration



## Visualize

In [ ]:
is_long = False
if is_long:
    resample_period = "60min"
    df = get_weather_data(date_from="2020-01-01", date_to="2020-12-31", resample_period=resample_period)
else:
    resample_period = "15min"
    df = get_weather_data(date_from="2020-07-01", date_to="2020-07-02", resample_period=resample_period)

In [ ]:
time_period = pd.to_timedelta(resample_period).total_seconds()

greenhouse = Greenhouse(
    time_period = time_period,
)

In [ ]:
for timestamp, row in df.iterrows():
    results = greenhouse.run(timestamp, row)

    for col in results.keys():
            df.loc[timestamp, col] = results[col]

In [ ]:
plot_multiline_dual_y(
    df.resample('D').mean() if is_long else df, 
    ["CO2_concentration_ppm"],
    ["CO2_assimilation_rate_mol_per_s"],
    width=900, 
    height=235, 
    title="", 
    y_labels=["ppm", "mol / s"], 
    legend_label="", 
    date_format="",
    colors=["red", "orange"]
)

In [ ]:
print(f"Total Net Energy: {round(df['net_energy_kWh'].sum(), 2)} kWh")
print(f"Total Wasted PAR: {round(df['wasted_PAR_total_umol'].sum() * 1e6, 2)} mol")

plot_multiline(
    df.resample('D').mean() if is_long else df, 
    ["total_energy_used_kWh", "total_energy_generated_kWh", "net_energy_kWh"],
    width=900, 
    height=235, 
    title="", 
    y_label="kWh", 
    legend_label="", 
    date_format=""
) & \
plot_multiline(
    df.resample('D').mean() if is_long else df, 
    ["actual_PPFD_umol_per_m2_s", "wasted_PPFD_umol_per_m2_s", "natural_PPFD"],
    width=900, 
    height=235, 
    title="", 
    y_label="umol / m2 / s", 
    legend_label="", 
    date_format=""
)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f5dc715-67f7-4c8c-98f7-a87b736d3338' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>